In [164]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 분할
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV # 그리드 서치(최적의 파라미터를 찾는것..?)

# 모델
from sklearn.linear_model import LinearRegression # 1
from sklearn.tree import DecisionTreeRegressor, plot_tree # 2
from sklearn.ensemble import RandomForestRegressor # 3
from xgboost import XGBRegressor # 4
from xgboost import plot_importance
from sklearn.neighbors import KNeighborsRegressor # 5
from sklearn.svm import SVR # 6

# 평가 지표
from sklearn.metrics import mean_squared_error 

## 1단계: 기존 데이터로 모델 학습

In [165]:
df = pd.read_csv("data.csv")
df.head()

,wave,family_member,gender,year_born,education_level,religion,company_size,log_income,region_2,region_3,...,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9,reason_none_worker_1,reason_none_worker_3,reason_none_worker_7,reason_none_worker_9,reason_none_worker_10
0,1,1,2,1940,2,1,1,6.131226,0,1,...,0,1,0,0,0,0,0,0,0,0
1,14,3,1,1939,5,2,0,7.446001,0,0,...,0,0,0,0,0,0,0,0,1,0
2,8,3,1,1967,6,1,1,7.956477,1,0,...,1,0,0,0,0,0,0,0,0,0
3,7,5,1,1947,3,1,1,8.364042,0,0,...,0,1,0,0,0,0,0,0,0,0
4,13,2,1,1940,2,2,0,7.382746,0,0,...,0,0,0,0,0,0,0,0,0,1


In [166]:
# 라벨 데이터를 분리
data = df.drop('log_income', axis=1) 
label = df.log_income

In [167]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

In [168]:
# 모델 선정 및 학습
model_xgb=XGBRegressor(gamma= 0, learning_rate= 0.1, max_depth= 10, n_estimators= 200, n_jobs=-1)
model_xgb.fit(X_train, y_train)                               

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

## 2단계: submission 데이터 전처리

In [169]:
sub = pd.read_csv("submission.csv", index_col=0) # 원본 데이터
new_data = sub.copy()
new_data.head()

,id,year,wave,region,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker,income
51802,47520101,2015,11,2,1,1,1965,5,3,2,873,1,,0
24685,23120101,2015,11,2,4,1,1970,5,1,2,521,1,,0
20910,19780101,2009,5,4,3,1,1946,5,1,2,,,10,0
24267,22730101,2005,1,2,4,1,1968,7,1,2,9999,10,,0
71797,63660101,2009,5,4,2,1,1942,3,1,1,611,1,,0


In [170]:
len(new_data)

13929

In [171]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13929 entries, 51802 to 23628
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  13929 non-null  int64 
 1   year                13929 non-null  int64 
 2   wave                13929 non-null  int64 
 3   region              13929 non-null  int64 
 4   family_member       13929 non-null  int64 
 5   gender              13929 non-null  int64 
 6   year_born           13929 non-null  int64 
 7   education_level     13929 non-null  int64 
 8   marriage            13929 non-null  int64 
 9   religion            13929 non-null  int64 
 10  occupation          13929 non-null  object
 11  company_size        13929 non-null  object
 12  reason_none_worker  13929 non-null  object
 13  income              13929 non-null  int64 
dtypes: int64(11), object(3)
memory usage: 1.6+ MB


In [172]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75522 entries, 0 to 75521
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   wave                   75522 non-null  int64
 1   family_member          75522 non-null  int64
 2   gender                 75522 non-null  int64
 3   year_born              75522 non-null  int64
 4   education_level        75522 non-null  int64
 5   religion               75522 non-null  int64
 6   company_size           75522 non-null  int64
 7   region_2               75522 non-null  int64
 8   region_3               75522 non-null  int64
 9   region_4               75522 non-null  int64
 10  region_5               75522 non-null  int64
 11  region_6               75522 non-null  int64
 12  region_7               75522 non-null  int64
 13  marriage_2             75522 non-null  int64
 14  marriage_3             75522 non-null  int64
 15  marriage_4             75522 non-nul

In [173]:
columns = ['occupation', 'company_size', 'reason_none_worker']
for x in columns:
    new_data[x] = new_data[x].apply(lambda x: x.strip())    # 해당 컬럼에 공백을 제거하고
    new_data.loc[new_data[x] == '', x] = np.NaN             # 빈칸을 갖고 있는 행을 추출하여 해당 칼럼에 NaN을 넣는다

### occupation 처리 

In [174]:
new_data['occupation'] = new_data['occupation'].apply(lambda x : x[0] if type(x) == str else x)

In [175]:
new_data.occupation.value_counts()

occupation
6    1745
9    1684
8    1089
3     958
7     939
2     847
5     668
4     608
1     400
Name: count, dtype: int64

In [176]:
new_data.loc[(new_data.reason_none_worker == '0') | (new_data.reason_none_worker == '99'), 'reason_none_worker'] = 11

In [177]:
new_data.loc[(new_data.company_size.notnull()) & (new_data.occupation.isnull()), 'company_size'] = 0

In [178]:
new_data.loc[new_data.company_size == '99', 'company_size'] = 1

In [179]:
new_data.fillna(0, inplace=True) # 3. 결측치(NaN)은 0으로 대체한다

In [180]:
new_data.info() # 아직 occupation, company_size, reason_none_worker이 int형이 아니다

<class 'pandas.core.frame.DataFrame'>
Index: 13929 entries, 51802 to 23628
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  13929 non-null  int64 
 1   year                13929 non-null  int64 
 2   wave                13929 non-null  int64 
 3   region              13929 non-null  int64 
 4   family_member       13929 non-null  int64 
 5   gender              13929 non-null  int64 
 6   year_born           13929 non-null  int64 
 7   education_level     13929 non-null  int64 
 8   marriage            13929 non-null  int64 
 9   religion            13929 non-null  int64 
 10  occupation          13929 non-null  object
 11  company_size        13929 non-null  object
 12  reason_none_worker  13929 non-null  object
 13  income              13929 non-null  int64 
dtypes: int64(11), object(3)
memory usage: 1.6+ MB


In [181]:
# 4. occupation, company_size, reason_none_worker을 int형으로 변환
columns = ['occupation', 'company_size', 'reason_none_worker']
for x in columns:
    new_data[x] = new_data[x].apply(lambda x: int(x))

In [182]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13929 entries, 51802 to 23628
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   id                  13929 non-null  int64
 1   year                13929 non-null  int64
 2   wave                13929 non-null  int64
 3   region              13929 non-null  int64
 4   family_member       13929 non-null  int64
 5   gender              13929 non-null  int64
 6   year_born           13929 non-null  int64
 7   education_level     13929 non-null  int64
 8   marriage            13929 non-null  int64
 9   religion            13929 non-null  int64
 10  occupation          13929 non-null  int64
 11  company_size        13929 non-null  int64
 12  reason_none_worker  13929 non-null  int64
 13  income              13929 non-null  int64
dtypes: int64(14)
memory usage: 1.6 MB


In [183]:
new_data.loc[((new_data.marriage == 0) | (new_data.marriage == 9)) & (new_data.year - new_data.year_born <= 18), 'marriage'] = 1 

In [184]:
new_data.loc[((new_data.marriage == 0) | (new_data.marriage == 9)) & (new_data.family_member == 1), 'marriage'] = 3

In [185]:
# 마지막으로 남은 데이터는 나이도 그리 많지 않고 가족 수도 많기 때문에 기혼을 뜻하는 2로 변경한다
new_data.loc[(new_data.marriage == 0) | (new_data.marriage == 9), 'marriage'] = 2

In [186]:
new_data.marriage.value_counts()

marriage
1    8671
2    3080
3    1161
5     820
4     192
6       5
Name: count, dtype: int64

In [187]:
new_data.loc[(new_data.marriage == 1) & (new_data.year-new_data.year_born > 18) & (new_data.family_member >= 2), 'marriage'] = 2

In [188]:
new_data.loc[(new_data.marriage == 1) & (new_data.year-new_data.year_born > 18) & (new_data.family_member == 1), 'marriage'] = 6

In [189]:
new_data.loc[new_data.religion == 9, 'religion'] = 1

In [190]:
new_data.loc[new_data.family_member > 5, 'family_member'] = 5

In [191]:
new_data = new_data.drop(['id', 'year', 'income'], axis=1)

In [192]:
new_data.reason_none_worker.value_counts()

reason_none_worker
0     9177
10    2868
8      633
9      391
1      351
7      161
6      153
4       95
11      52
5       41
3        7
Name: count, dtype: int64

In [193]:
new_data.loc[(new_data.reason_none_worker == 2) | (new_data.reason_none_worker == 4), 'reason_none_worker'] = 1
new_data.loc[new_data.reason_none_worker == 8, 'reason_none_worker'] = 7
new_data.loc[(new_data.reason_none_worker == 5) | (new_data.reason_none_worker == 6) | (new_data.reason_none_worker == 11), 'reason_none_worker'] = 3

In [194]:
new_data.reason_none_worker.value_counts()

reason_none_worker
0     9177
10    2868
7      794
1      446
9      391
3      253
Name: count, dtype: int64

In [195]:
# One-Hot Encoding : region, marriage, occupation, reason_none_worker, 
new_data_encoded = pd.get_dummies(new_data, columns=['region', 'marriage','occupation', 'reason_none_worker'], drop_first=True, dtype=int)
new_data_encoded.head()

,wave,family_member,gender,year_born,education_level,religion,company_size,region_2,region_3,region_4,...,occupation_5,occupation_6,occupation_7,occupation_8,occupation_9,reason_none_worker_1,reason_none_worker_3,reason_none_worker_7,reason_none_worker_9,reason_none_worker_10
51802,11,1,1,1965,5,2,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
24685,11,4,1,1970,5,2,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
20910,5,3,1,1946,5,2,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
24267,1,4,1,1968,7,2,10,1,0,0,...,0,0,0,0,1,0,0,0,0,0
71797,5,2,1,1942,3,1,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [196]:
new_data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13929 entries, 51802 to 23628
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   wave                   13929 non-null  int64
 1   family_member          13929 non-null  int64
 2   gender                 13929 non-null  int64
 3   year_born              13929 non-null  int64
 4   education_level        13929 non-null  int64
 5   religion               13929 non-null  int64
 6   company_size           13929 non-null  int64
 7   region_2               13929 non-null  int32
 8   region_3               13929 non-null  int32
 9   region_4               13929 non-null  int32
 10  region_5               13929 non-null  int32
 11  region_6               13929 non-null  int32
 12  region_7               13929 non-null  int32
 13  marriage_2             13929 non-null  int32
 14  marriage_3             13929 non-null  int32
 15  marriage_4             13929 non-null

In [197]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75522 entries, 0 to 75521
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   wave                   75522 non-null  int64
 1   family_member          75522 non-null  int64
 2   gender                 75522 non-null  int64
 3   year_born              75522 non-null  int64
 4   education_level        75522 non-null  int64
 5   religion               75522 non-null  int64
 6   company_size           75522 non-null  int64
 7   region_2               75522 non-null  int64
 8   region_3               75522 non-null  int64
 9   region_4               75522 non-null  int64
 10  region_5               75522 non-null  int64
 11  region_6               75522 non-null  int64
 12  region_7               75522 non-null  int64
 13  marriage_2             75522 non-null  int64
 14  marriage_3             75522 non-null  int64
 15  marriage_4             75522 non-nul

In [198]:
y_pred = model_xgb.predict(new_data_encoded)

In [199]:
sub['income'] = np.exp(y_pred)

In [200]:
sub.head()

,id,year,wave,region,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker,income
51802,47520101,2015,11,2,1,1,1965,5,3,2,873,1,,2804.934814
24685,23120101,2015,11,2,4,1,1970,5,1,2,521,1,,5644.544922
20910,19780101,2009,5,4,3,1,1946,5,1,2,,,10,3551.990479
24267,22730101,2005,1,2,4,1,1968,7,1,2,9999,10,,4152.091797
71797,63660101,2009,5,4,2,1,1942,3,1,1,611,1,,1918.995117


In [201]:
df.to_csv("submission3.csv", index=False)